In [1]:
from elasticsearch import Elasticsearch
import pandas as pd
from collections import OrderedDict

from bert_server_client import BertServerClient

pd.set_option('display.max_colwidth', None)

bsc = BertServerClient(bert_client_ip="bertserver")
es = Elasticsearch('elasticsearch:9200')

In [2]:
def search(query, size=10):
    query_vector = bsc.sentence2vec([query])[0]
    script_query = {
        "script_score": {
            "query": {"match_all": {}},
            "script": {
                "source": "(cosineSimilarity(params.query_vector, doc['sentence_vector']) + 1)/2.0",
                "params": {"query_vector": query_vector},
            },
        }
    }

    response = es.search(
        index="vector_search",
        body={
            "size": size,
            "query": script_query,
            "_source": {"includes": ["sentence"]},
        },
    )

    return pd.DataFrame([
            OrderedDict({
                'sentence': row['_source']['sentence'], 
                'score': row['_score']
            }) for _, row in pd.DataFrame(response['hits']['hits']).iterrows()])

In [6]:
search("お金が欲しい", size=20)

/usr/local/lib/python3.7/site-packages/bert_serving/client/__init__.py:290: UserWarning: server does not put a restriction on "max_seq_len", it will determine "max_seq_len" dynamically according to the sequences in the batch. you can restrict the sequence length on the client side for better efficiency
  warnings.warn('server does not put a restriction on "max_seq_len", '


,sentence,score
0,結論から申し上げますと、\n就職活動で\n0デベロッパー業界の中でグループ会社まで受ける\n,0.936101
1,webテ勉強、ob訪問も行う。\n,0.934401
2,peファンドは難しいですか?,0.933944
3,twitterがモンスター製造機であるという説を詳しく教えてください。,0.933405
4,グルメ関連の回答はいつもスクショを撮り参考にさせてもらってます笑\n\n,0.933288
5,転職などによって流入する側ですか?,0.932817
6,貴重な新卒入社の機会を、\n0ステータス待遇面を少し落としてでもデベロッパーのグループ会社まで検討する\n0幅広い業界のトップ企業を検討する\n\nそれぞれについてどのように思われますか?,0.932718
7,同業他社\nコンサルティングファーム\n投資銀行\nファンド\n事業会社のm&a推進部門、経営企画経営戦略部門\nと、就職転職情報サイトに書かれていましたが、それぞれの割合をベラージオさんの感覚値でも教えて頂きたく、お願いいたします。,0.932484
8,事業会社とのプロジェクトが終了した後も、プライベート等でお付き合いを継続する人は多いですか?,0.932361
9,個人でアルゴリズムトレードを行なっているものです。\n,0.932274
